# Advent of Code 2022 - Day 9

In [20]:
from pathlib import Path
from typing import Final, List, Dict, Tuple, Set, Any, TypeAlias

## Part One

- [X] Read input
- [ ] Split each instruction to a Tuple of (Direction, Number)
- [ ] Create a grid for the rope to move on?
- [ ] Move Head
- [ ] Compare Tail position to head and move accordingly
- [ ] Mark position of tail on a boolean grid (True is tail has passed that space)
- [ ] Count grid


In [21]:
INPUT_NAME : str = "day9_input_test.txt"
MovePosition : TypeAlias = Tuple[int, int]

In [8]:
def read_input(input_name: str) -> List[str]:
    input_path : Path = Path.cwd() / "input" / input_name

    with open(input_path, "r") as input_file:

        lines : List = input_file.readlines()

    return [line.rstrip("\n") for line in lines]

In [9]:
instructions_input : List[MovePosition]= [tuple(line.split(" ")) for line in read_input(INPUT_NAME)]

In [40]:

STARTING_POSITION : Final[MovePosition]= (0,0)
STEP = 1
ROW = 0
COL = 1

In [13]:
covered_positions : List[MovePosition] = []

In [47]:
def make_step_move(current_position: MovePosition, move: MovePosition, is_tail: bool = False) -> MovePosition:
    """
    Co-ordinates are relative to start (0,0), as (row, col)
    """  
    
    return (current_position[ROW] + move[ROW], current_position[COL] + move[COL])
    

In [48]:
def move_head(current_position: MovePosition, move: MovePosition) -> MovePosition:
    
    return make_step_move(current_position, move)

In [49]:
def move_tail(direction: str, head_position: MovePosition, current_position: MovePosition) -> MovePosition:
    """
    Delta position: -ve means below or to the left of head
    """
    
    delta_position = (current_position[ROW]-head_position[ROW], current_position[COL]-head_position[COL])
    
    # Check if in same row/col (else diagonal move)
    # Both checks will be false if diagonal move required
    if delta_position[ROW] == head_position[ROW] or delta_position[COL] == head_position[COL]:
        # Move orthogonally
        move = generate_step_move(direction)
        return make_step_move(current_position, move)
    else:
        # Move diagonally
        ...
    

In [50]:
def generate_step_move(direction: str) -> MovePosition:
    
    step = -1 if direction in ["L", "D"] else 1
    
    if direction in ["L", "R"]:
        move : MovePosition = (0, step)
    elif direction in ["U", "D"]:
        move = (step, 0)
        
    return move

In [53]:
def make_move(
    direction: str, count: str, head_position: MovePosition, tail_position: MovePosition
) -> Tuple[MovePosition, List[MovePosition]]:
    """
    Returns current head position, and list of positions tail has been in for this move
    """

    count = int(count)

    current_head_position = head_position
    current_tail_position = tail_position
    covered_positions = []

    for idx in range(count):
        move = generate_step_move(direction)
        current_head_position = move_head(current_head_position, move)
        # current_tail_position = move_tail(direction, current_head_position, current_tail_position)
        
    return current_head_position
        